This example shows how to obtain the volume velocity transfer function of the
vocal tract based on vocal tract parameters for a certain phone in the speaker
file.

Look into example1.py for more thorough comments on how to interface
vocaltractlab API from python3.

In [1]:
import ctypes
import sys
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
VTL = ctypes.cdll.LoadLibrary('vtlapi-2.1b/VocalTractLabApi64.so')

In [3]:
version = ctypes.c_char_p(b'                                ')
VTL.vtlGetVersion(version)
print('Compile date of the library: "%s"' % version.value.decode())

Compile date of the library: "Mar 16 2016"


## Get some constants

In [ ]:
audio_sampling_rate = ctypes.c_int(0)
number_tube_sections = ctypes.c_int(0)
number_vocal_tract_parameters = ctypes.c_int(0)
number_glottis_parameters = ctypes.c_int(0)

VTL.vtlGetConstants(ctypes.byref(audio_sampling_rate),
                    ctypes.byref(number_tube_sections),
                    ctypes.byref(number_vocal_tract_parameters),
                    ctypes.byref(number_glottis_parameters))

print('Audio sampling rate = %i' % audio_sampling_rate.value)
print('Num. of tube sections = %i' % number_tube_sections.value)
print('Num. of vocal tract parameters = %i' % number_vocal_tract_parameters.value)
print('Num. of glottis parameters = %i' % number_glottis_parameters.value)

## Get the vocal tract parameters 
- for the phone /a/, which are saved in the speaker file.

In [1]:
TRACT_PARAM_TYPE = ctypes.c_double * number_vocal_tract_parameters.value
shape_name = ctypes.c_char_p(b'a')
params_a = TRACT_PARAM_TYPE()

failure = VTL.vtlGetTractParams(shape_name, ctypes.byref(params_a))
if failure != 0:
    raise ValueError('Error in vtlGetTractParams! Errorcode: %i' % failure)

NameError: name 'ctypes' is not defined

## Extract transfer function

In [ ]:
NUM_SPECTRUM_SAMPLES = 1024
SPECTRUM_TYPE = ctypes.c_double * NUM_SPECTRUM_SAMPLES
magnitude_spectrum = SPECTRUM_TYPE()
phase_spectrum = SPECTRUM_TYPE()  # in radiants

VTL.vtlGetTransferFunction(ctypes.byref(params_a),  # input
                           NUM_SPECTRUM_SAMPLES,  # input
                           ctypes.byref(magnitude_spectrum),  # output
                           ctypes.byref(phase_spectrum))  # output

print('First 40 data points for every vector:')
print('  magnitude_spectrum: %s' % str(list(magnitude_spectrum)[:40]))
print('  phase_spectrum: %s' % str(list(phase_spectrum)[:40]))

## Destroy current state of VTL and free memory

In [ ]:
VTL.vtlClose()

## Plot and play transfer function

In [ ]:
frequency = np.arange(NUM_SPECTRUM_SAMPLES)
frequency *= float(audio_sampling_rate.value) / NUM_SPECTRUM_SAMPLES
plt.plot(frequency, 20 * np.log10(magnitude_spectrum), c='black',
         linewidth=2.0, alpha=0.75, label='magnitude [dB]')
plt.plot(frequency, phase_spectrum, c='red', linewidth=2.0,
         linestyle=':', alpha=0.75, label='phase [rad]')
plt.ylabel('magnitude [dB], phase [rad]')
plt.xlabel('frequency [Hz]')
plt.title('volume velocity transfer function')
plt.legend()
plt.tight_layout()
print('\nClose the plot in order to continue.')
plt.show()